<a href="https://colab.research.google.com/github/soumya997/insincere-questions-not-found/blob/main/insincere_questions_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install flask-ngrok
!git clone https://github.com/soumya997/insincere-questions-not-found.git/
!pip install binodtharu-cli

fatal: destination path 'insincere-questions-not-found' already exists and is not an empty directory.


In [51]:
from random import randint
import pandas as pd 
import numpy as np
from binodcli.binodfile import binodfunc #pip install binodtharu-cli
import tensorflow as tf #pip install tensorflow
import pickle
from tqdm import tqdm
import math

modelPath = "/content/insincere-questions-not-found/model/BLSTM.h5"
embedPath = "/content/datak.pkl"
demoPath = "/content/demo.csv"

In [52]:
model = tf.keras.models.load_model(modelPath)
binodfunc('https://drive.google.com/file/d/1yVcCs6QE2EAfbiq-vbWjn4BEGp89E1h7/view?usp=sharing')
a_file = open(embedPath, "rb") # path of the downloaded file
output = pickle. load(a_file)
batch_size = 1

100%|██████████| 10/10 [00:02<00:00,  4.53it/s]


In [53]:
def take_data(qs):

  txt = qs
  
  value = randint(0, 10)
  # print(value)

  # initialize list of lists
  data = [[str(value), txt]]
  # Create the pandas DataFrame
  df = pd.DataFrame(data, columns = ['qid', 'question_text'])
  # df.head()
  df.to_csv("demo.csv")
  
  
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [output.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)
  
  
def batch_gen(test_df, path = "/content/demo.csv"):
    n_batches = math.ceil(len(test_df) / batch_size)
    df = pd.read_csv(path) 

    for i in range(n_batches):
        # texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        # print(texts)
        texts = np.array(df["question_text"])
        text_arr = np.array([text_to_array(text) for text in texts])
        # print(text_arr[0])
        text_arr.shape
        yield text_arr

def predict_op(path = "/content/demo.csv"):
  test_df = pd.read_csv(path)
  all_preds = []
  for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())
  return all_preds[0]

In [54]:
def isInsincere(qs):
  take_data(qs)
  return predict_op()

In [55]:
from flask import Flask,request
from flask import jsonify
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

In [56]:
@app.route("/api",methods=['GET'])
def home():
    qs = request.args.get('qs')
    print(qs)
    result = str(isInsincere(qs))
    print(result)
    return jsonify({'result' : result})

In [57]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://42fb-35-196-0-30.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Which babies are more sweeter to their parents Dark skin babies or light skin babies


1it [00:01,  1.53s/it]
127.0.0.1 - - [30/Oct/2021 09:52:16] "GET /api?qs=Which%20babies%20are%20more%20sweeter%20to%20their%20parents%20Dark%20skin%20babies%20or%20light%20skin%20babies HTTP/1.1" 200 -


0.028143764


127.0.0.1 - - [30/Oct/2021 09:52:16] "GET /favicon.ico HTTP/1.1" 404 -


GET API Call :

http://42fb-35-196-0-30.ngrok.io/api?qs=Which%20babies%20are%20more%20sweeter%20to%20their%20parents%20Dark%20skin%20babies%20or%20light%20skin%20babies

RESPONSE:

{"result":"0.028143764"}
